In [1]:
from models.resnet import ResNet18
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.utils import shuffle

from utils.sorting_utils import sort_one_class, sort_all_classes, weighted_highest_sampling

class DataLoader():
    
    def __init__(self, root='./data', batch_size=64):
        self.root = root        
        self.trainset = None
        self.testset = None
        self.known_strategies = ['freeze', 'shuffle', 'homogeneous', 'heterogeneous', 'max_k_loss', 'min_k_loss']
        self.seed_incrementer = {strategy: 0 for strategy in self.known_strategies}
        
        self.batch_size = batch_size
        
    def download_cifar(self):
        #print('==> Preparing data..')
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        # data needs to be loaded through dataloader to get into the correct format
        trainset = torchvision.datasets.CIFAR10(root=self.root, train=True, download=True, transform=transform_train)
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=self.batch_size, shuffle=False, num_workers=2)
        self.X_train, self.Y_train = [], []
        for x, y in trainloader:
            self.X_train.extend(x.numpy()) #numpy needed for a casting workaround
            self.Y_train.extend(y.numpy())
        self.X_train = np.array(self.X_train)
        self.Y_train = np.array(self.Y_train)
        
        testset = torchvision.datasets.CIFAR10(root=self.root, train=False, download=True, transform=transform_test)
        testloader = torch.utils.data.DataLoader(testset, batch_size=self.batch_size, shuffle=False, num_workers=2)
        self.X_test, self.Y_test = [], []
        for x, y in testloader:
            self.X_test.extend(x.numpy())
            self.Y_test.extend(y.numpy())
        self.X_test = np.array(self.X_test)
        self.Y_test = np.array(self.Y_test)
                
        self.X_batches_train, self.Y_batches_train = None, None
        self.X_batches_test, self.Y_batches_test = None, None
            
    def yield_batches(self, strategy, use_train=True, random_state=None, criterion=None, device=None, num_iterations=None):
        assert strategy in self.known_strategies, 'Unknown action'
        
        # seeds will be incremented at each epoch, to give the shuffling methods a new random seed
        # this will still be deterministic!
        current_seed = random_state + self.seed_incrementer[strategy]
        self.seed_incrementer[strategy] += 1
        
        # prepare the data
        if strategy == 'freeze':
            self.X_batches_train, self.Y_batches_train = self.X_train, self.Y_train
            self.X_batches_test, self.Y_batches_test = self.X_test, self.Y_test
            yield_batchwise = True
        elif strategy == 'shuffle':
            self.X_batches_train, self.Y_batches_train = shuffle(self.X_train, self.Y_train, random_state=current_seed)
            self.X_batches_test, self.Y_batches_test = shuffle(self.X_test, self.Y_test, random_state=current_seed)
            yield_batchwise = True
        elif strategy == 'homogeneous':
            self.X_batches_train, self.Y_batches_train = sort_one_class(self.X_train, self.Y_train, self.batch_size, \
                                                            use_shuffle=True, random_state=current_seed)
            self.X_batches_test, self.Y_batches_test = sort_one_class(self.X_test, self.Y_test, self.batch_size, \
                                                            use_shuffle=True, random_state=current_seed)
            yield_batchwise = True
        elif strategy == 'heterogeneous':
            self.X_batches_train, self.Y_batches_train = sort_all_classes(self.X_train, self.Y_train, self.batch_size, \
                                                              use_shuffle=True, random_state=current_seed)
            self.X_batches_test, self.Y_batches_test = sort_all_classes(self.X_test, self.Y_test, self.batch_size, \
                                                              use_shuffle=True, random_state=current_seed)
            yield_batchwise = True
        elif strategy == 'max_k_loss':
            self._initialize_weights(criterion, device, seed=random_state)
            
            for iteration in range(num_iterations):
                pulled_idxs = weighted_highest_sampling(self.weighted_indices, batch_size=self.batch_size, top_fn=max)
                yield self.get_from_idxs(pulled_idxs)
                self._update_weights(pulled_idxs, criterion, device)
                
            # set to False to stop method after this yielding
            yield_batchwise = False

        elif strategy == 'min_k_loss':
            self._initialize_weights(criterion, device, seed=random_state)
            
            for iteration in range(num_iterations):
                pulled_idxs = weighted_highest_sampling(self.weighted_indices, batch_size=self.batch_size, top_fn=min)
                yield self.get_from_idxs(pulled_idxs)
                self._update_weights(pulled_idxs, criterion, device)
                
            # set to False to stop method after this yielding
            yield_batchwise = False
            
        if yield_batchwise:
            # yield it in batches
            batch_idx = 0
            X = self.X_batches_train if use_train else self.X_batches_test
            Y = self.Y_batches_train if use_train else self.Y_batches_test
            X, Y = torch.from_numpy(X), torch.from_numpy(Y)
            while batch_idx < len(X):
                yield X[batch_idx: batch_idx+self.batch_size], Y[batch_idx: batch_idx+self.batch_size]
                batch_idx += self.batch_size
            
    def get_from_idxs(self, idxs, use_train=True):
        if use_train:
            X, Y = self.X_train[idxs], self.Y_train[idxs]
        else:
            X, Y = self.X_test[idxs], self.Y_test[idxs]
        return torch.from_numpy(X), torch.from_numpy(Y)
    
    def set_model(self, model):
        self.model = model
    
    def _initialize_weights(self, criterion, device, seed=None):
        assert self.model != None, 'Model needs to be set first!'
        
        self.weighted_indices = {}
        sample_idx = 0
        
        # using freeze as this is the simplest way of getting data in batches fast
        for inputs, targets in dataloader.yield_batches('freeze', random_state=seed, use_train=True):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = self.model(inputs)
            losses = criterion(outputs, targets)
            for loss in losses:
                loss = float(loss.cpu().detach().numpy())
                self.weighted_indices[sample_idx] = loss
                sample_idx += 1
                
    def _update_weights(self, idxs, criterion, device, seed=None):
        inputs, targets = self.get_from_idxs(idxs)
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = self.model(inputs)
        losses = criterion(outputs, targets)
        for idx, loss in zip(idxs, losses):
            self.weighted_indices[idx] = loss

In [3]:
dataloader = DataLoader()
dataloader.download_cifar()

Files already downloaded and verified
Files already downloaded and verified


In [4]:
net = ResNet18()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = net.to(device)
criterion = nn.CrossEntropyLoss(reduction='none')

In [5]:
dataloader.set_model(net)

In [8]:

for batch_idx, (inputs, targets) in enumerate(
                dataloader.yield_batches(
                    strategy, random_state=seed, use_train=True, criterion=criterion, device=device, num_iterations=10)
                ):
    print(batch_idx)
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = net(inputs)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


KeyboardInterrupt: 